<h3><b>1. Scrapping Data 50x3 Artikel</b></h3>


In [ ]:
# Install necessary libraries (if not already installed)
!pip install beautifulsoup4 requests pandas -q
!pip install openpyxl -q

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Read URLs from an Excel file (replace 'urls.xlsx' with your file path)
excel_path = 'data/artikel_damri.xlsx'  
df_urls = pd.read_excel(excel_path, engine='openpyxl')

# If the sheet has a column named 'url' use it, otherwise use the first column
if 'link' in [c.lower() for c in df_urls.columns]:
    # find the actual column name matching 'url' (case-insensitive)
    url_col = next(c for c in df_urls.columns if c.lower() == 'link')
    urls = df_urls[url_col].dropna().astype(str).tolist()
else:
    urls = df_urls.iloc[:, 0].dropna().astype(str).tolist()

# Function to scrape content from a URL
def scrape_content(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract the title of the article
            title = soup.find('title').text

            # Extract the article's content
            paragraphs = soup.find_all('p')
            content = "\n".join([para.text for para in paragraphs])

            return {
                "url": url,
                "title": title,
                "content": content
            }
        else:
            return {
                "url": url,
                "title": None,
                "content": None,
                "error": f"Failed to fetch page, status code: {response.status_code}"
            }
    except Exception as e:
        return {
            "url": url,
            "title": None,
            "content": None,
            "error": str(e)
        }

# Scrape each URL and store the results in a list of dictionaries
data = []
for url in urls:
    result = scrape_content(url)
    data.append(result)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

# Save the DataFrame to a CSV file (optional)
df.to_csv('data/scraped_articles.csv', index=False)


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


<h3><b>2. Lakukan EDA untuk Dataset Scraping</b></h3>

In [ ]:
!pip install pandas matplotlib seaborn nltk
!pip install Sastrawi

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import nltk
import string

nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


# Create stemmer and lemmatizer objects
factory = StemmerFactory()
stemmer = factory.create_stemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Case lowering
    text = text.lower()

    # Punctuation removal
    text = re.sub(r'[^\w\s]', '', text)

    # Stemming
    text = stemmer.stem(text)

    # Lemmatization (requires a part-of-speech tag for optimal results, using 'v' for verb as a default)
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word, 'v') for word in words]
    text = ' '.join(lemmatized_words)

    return text

# Apply the cleaning function to the 'review' column
df['cleaned_review'] = df['content'].apply(clean_text)

# Display the DataFrame with the cleaned reviews
print(df_jknrev[['review', 'cleaned_review']])